# Muon transport
#### Roland Grinis  -  Researcher at MIPT Nuclear Physics Methods lab  -  CTO at GrinisRIT (grinisrit.com)

Code available within NOA github.com/grinisrit/noa - Bayesian computation algorithms in C++17 over LibTorch

## Installation

The `conda` environment provided with the repository has all the required dependencies. For this particular tutorial we will need the following `python` packages:

In [1]:
import torch
from torch.utils.cpp_extension import load

Now we need to build and load `C++17/CUDA` extensions for `PyTorch`, set up the locations:

In [3]:
!mkdir -p build
noa_location = '../..'

If you are running this on Google Colab, you need to clone `NOA` and set `noa_location` accordingly:
```python
!git clone https://github.com/grinisrit/noa.git
noa_location = 'noa'
```

Also, make sure that `ninja` and `g++-8` or higher are available. The following commands will do that for you:
```python
!pip install Ninja
!apt install gcc-8 g++-8 -y
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 1000
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-8 1000
!gcc --version
!g++ --version
!nvcc --version
```
Finally, you get the extensions into `python` by calling `load`:

In [5]:
muons = load(name='muons',
             build_directory='./build',
             sources=[f'{noa_location}/docs/pms/muons.cc'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             extra_ldflags=['-lstdc++fs'],
             verbose=False)

In [6]:
muons_cuda = load(name='muons_cuda',
             build_directory='./build',       
             sources=[f'{noa_location}/docs/pms/muons.cu'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             extra_cuda_cflags=['-std=c++17 --extended-lambda'],
             extra_ldflags=['-lstdc++fs'],
             verbose=False) if torch.cuda.is_available() else None

## Differential Cross-Sections calculations

Differential cross-sections are implemented in `<noa/pms/dcs.hh>` for `CPU` and `<noa/pms/dcs.cuh>` for `CUDA`. Here, we demonstrate the calculations for muons passing through the standard rock. In `<noa/pms/constants.hh>` you will find:
```cpp
constexpr ParticleMass MUON_MASS = 0.10565839;     // GeV/c^2

constexpr AtomicElement<Scalar> STANDARD_ROCK =
            AtomicElement<Scalar>{
                    22.,       // Atomic mass in g/mol
                    0.1364E-6, // Mean Excitation in GeV
                    11 // Atomic number
    };
```

Let's get a range of kinetic and recoil energies:

In [7]:
kinetic_energies = torch.linspace(1e-3, 1e6, 10000).double()
recoil_energies = 0.0505 * kinetic_energies

In [8]:
kinetic_energies_gpu = kinetic_energies.cuda()
recoil_energies_gpu = recoil_energies.cuda()

In [9]:
kinetic_energies_gpu[-5:]

tensor([ 999599.9375,  999700.0000,  999800.0000,  999900.0000, 1000000.0000],
       device='cuda:0', dtype=torch.float64)

```cpp
#include <noa/utils/common.hh>
#include <noa/pms/constants.hh>
#include <noa/pms/dcs.hh>

using namespace noa::pms;
using namespace noa::utils;

inline Tensor bremsstrahlung(Tensor kinetic_energies, Tensor recoil_energies) {
    const auto result = torch::zeros_like(kinetic_energies);
    dcs::vmap<Scalar>(dcs::pumas::bremsstrahlung)(
            result, kinetic_energies, recoil_energies, STANDARD_ROCK, MUON_MASS);
    return result;
}
```

In [26]:
brems = muons.bremsstrahlung(kinetic_energies, recoil_energies)
brems[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       dtype=torch.float64)

```cpp
#include <noa/pms/kernels.cuh>
#include <noa/pms/constants.hh>

using namespace noa::pms;
using namespace noa::utils;

inline torch::Tensor bremsstrahlung(torch::Tensor kinetic_energies, torch::Tensor recoil_energies) {
    const auto result = torch::zeros_like(kinetic_energies);
    dcs::pumas::cuda::vmap_bremsstrahlung(result, kinetic_energies, recoil_energies, STANDARD_ROCK, MUON_MASS);
    return result;
}
```

In [25]:
brems_gpu = muons_cuda.bremsstrahlung(kinetic_energies_gpu, recoil_energies_gpu);
brems_gpu[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       device='cuda:0', dtype=torch.float64)

In [13]:
(brems - brems_gpu.cpu()).abs().sum()

tensor(3.2179e-18, dtype=torch.float64)

In [14]:
%timeit muons_cuda.bremsstrahlung(kinetic_energies_gpu, recoil_energies_gpu);

155 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%timeit muons.bremsstrahlung(kinetic_energies, recoil_energies);

308 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


```cpp
dcs::vmap<Scalar>(dcs::pumas::pair_production)(
            result, kinetic_energies, recoil_energies, STANDARD_ROCK, MUON_MASS);
```

In [22]:
ppair = muons.pair_production(kinetic_energies, recoil_energies)
ppair[:5]

tensor([0.0000e+00, 6.5366e-06, 7.3699e-06, 7.7919e-06, 8.0572e-06],
       dtype=torch.float64)

```cpp
dcs::vmap<Scalar>(dcs::pumas::photonuclear)(
            result, kinetic_energies, recoil_energies, STANDARD_ROCK, MUON_MASS);
```

In [21]:
photonuc = muons.photonuclear(kinetic_energies, recoil_energies)
photonuc[:5]

tensor([0.0000e+00, 2.2912e-06, 2.1304e-06, 2.0719e-06, 2.0427e-06],
       dtype=torch.float64)

```cpp
dcs::vmap<Scalar>(dcs::pumas::ionisation)(
            result, kinetic_energies, recoil_energies, STANDARD_ROCK, MUON_MASS);
```

In [20]:
ionis = muons.ionisation(kinetic_energies, recoil_energies)
ionis[:5]

tensor([0.0000e+00, 3.0168e-05, 1.5300e-05, 1.0284e-05, 7.7575e-06],
       dtype=torch.float64)